In [2]:
import numpy as np
import pandas as pd
import seaborn as sb
import matplotlib.pyplot as plt
%matplotlib inline
plt.rcParams['figure.figsize'] = (16, 9)
plt.style.use('ggplot')
from sklearn import tree
from sklearn.metrics import accuracy_score
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from IPython.display import Image as PImage
from subprocess import check_call
from PIL import Image, ImageDraw, ImageFont

In [3]:
test_final_100k_ = pd.read_csv('Datos/test_final_100k.csv')

# Pre procesamiento de los datos


## Postulantes

In [4]:
postulantes_educacion_desde_15_Abril = pd.read_csv('Datos/Fiuba_desde_15_Abril/fiuba_1_postulantes_educacion.csv')
postulantes_genero_y_edad_desde_15_Abril = pd.read_csv('Datos/Fiuba_desde_15_Abril/fiuba_2_postulantes_genero_y_edad.csv')
postulantes_educacion_hasta_15_abril = pd.read_csv('Datos/fiuba_hasta_15_abril/fiuba_1_postulantes_educacion.csv')
postulantes_genero_y_edad_hasta_15_abril = pd.read_csv('Datos/fiuba_hasta_15_abril/fiuba_2_postulantes_genero_y_edad.csv')
postulantes_educacion_original = pd.read_csv('Datos/datos_navent_fiuba/fiuba_1_postulantes_educacion.csv')
postulantes_genero_y_edad_original = pd.read_csv('Datos/datos_navent_fiuba/fiuba_2_postulantes_genero_y_edad.csv')

In [5]:
postulantes_genero_y_edad = pd.concat([postulantes_genero_y_edad_desde_15_Abril,postulantes_genero_y_edad_hasta_15_abril,postulantes_genero_y_edad_original], axis=0)
postulantes_genero_y_edad = postulantes_genero_y_edad.drop_duplicates('idpostulante')

In [6]:
#Descarto los postulantes que están presentes en el set de prueba

postulantes_genero_y_edad = postulantes_genero_y_edad.set_index('idpostulante').drop(test_final_100k_.drop_duplicates('idpostulante').idpostulante).reset_index()

In [7]:
postulantes_educacion = pd.concat([postulantes_educacion_desde_15_Abril,postulantes_educacion_hasta_15_abril,postulantes_educacion_original], axis=0)
postulantes_educacion = postulantes_educacion

In [8]:
#Renombro las columnas a un nombre más apropiado

postulantes_educacion = postulantes_educacion.rename(columns={"nombre":"educacion"})

In [9]:
#Hay algun dato nulo?

postulantes_educacion.isnull().any()

idpostulante    False
educacion       False
estado          False
dtype: bool

In [10]:
#Para cada postulante, me interesa quedarme solamente con la mayor de sus educaciones

postulantes_educacion['Cod_edu'] = np.array([0 if e == 'Secundario' else 
                                             1 if e == 'Otro' else 
                                             2 if e == 'Terciario/Técnico' else 
                                             3 if e == 'Universitario' else 
                                             4 if e == 'Posgrado' else 
                                             5 if e == 'Master' else 6 for e in postulantes_educacion.educacion])

postulantes_educacion = postulantes_educacion.sort_values(['idpostulante','Cod_edu'], ascending = False).drop_duplicates('idpostulante')

In [11]:
postulantes_educacion.head()

,idpostulante,educacion,estado,Cod_edu
53093,zvxzzL6,Otro,Graduado,1
118722,zvxzvD5,Terciario/Técnico,En Curso,2
60154,zvxzrWm,Universitario,Graduado,3
50863,zvxzmED,Secundario,Graduado,0
211836,zvxzjoK,Terciario/Técnico,En Curso,2


In [12]:
#Ojo, fechas de nacimiento nulas

postulantes_genero_y_edad.isnull().any()

idpostulante       False
fechanacimiento     True
sexo               False
dtype: bool

In [13]:
#Convierto la columna fechanacimiento a formato datetime. 

postulantes_genero_y_edad['fechanacimiento'] = pd.to_datetime(postulantes_genero_y_edad['fechanacimiento'], errors='coerce')

In [14]:
#Corregir estas fechas nulas

postulantes_genero_y_edad.loc[postulantes_genero_y_edad.fechanacimiento.isnull()].head()

,idpostulante,fechanacimiento,sexo
1,8mW,NaT,NO_DECLARA
5,eRYllW,NaT,NO_DECLARA
6,52KB1p,NaT,NO_DECLARA
11,Y0mpjq,NaT,NO_DECLARA
14,a61YjW,NaT,NO_DECLARA


In [15]:
#Vamos ahora a agregar una columna adicional al dataframe, de modo que podamos observar la edad de cada persona

postulantes_genero_y_edad['edad'] = ((pd.datetime.today() - postulantes_genero_y_edad['fechanacimiento']).dt.days/365)
postulantes_genero_y_edad['edad'] = postulantes_genero_y_edad['edad'].fillna(0)
postulantes_genero_y_edad['edad'] = postulantes_genero_y_edad['edad'].astype(int)
postulantes_genero_y_edad.head(10)

,idpostulante,fechanacimiento,sexo,edad
0,YZP,1961-05-01,MASC,57
1,8mW,NaT,NO_DECLARA,0
2,aEWON,1967-06-16,FEM,50
3,Zjde31,1964-02-08,MASC,54
4,8wKqoM,1977-08-04,FEM,40
5,eRYllW,NaT,NO_DECLARA,0
6,52KB1p,NaT,NO_DECLARA,0
7,NA8o45,1974-08-16,MASC,43
8,eRYl5r,1975-10-11,FEM,42
9,8wKOlR,1977-12-24,MASC,40


In [16]:
#Reviso la edad de las personas que cumplen años el 29/02

postulantes_nacidos_en_febrero = postulantes_genero_y_edad.loc[postulantes_genero_y_edad['fechanacimiento'].dt.month == 2]
postulantes_nacidos_en_febrero.loc[postulantes_nacidos_en_febrero['fechanacimiento'].dt.day == 29].sort_values('fechanacimiento',ascending = False).head(10)

,idpostulante,fechanacimiento,sexo,edad
11375,VNr1dej,2000-02-29,MASC,18
437559,6rP38R3,2000-02-29,FEM,18
435820,3NP3dAJ,2000-02-29,FEM,18
431417,Bm5dkzR,2000-02-29,MASC,18
180285,96XkKlw,2000-02-29,FEM,18
138853,Yj5p1GX,2000-02-29,MASC,18
148087,1QP82xR,2000-02-29,FEM,18
197610,EzeN2Q4,2000-02-29,FEM,18
355768,JBraa0E,2000-02-29,MASC,18
333738,ekOwAWW,2000-02-29,FEM,18


In [17]:
#Merge de los datos de los postulantes

postulantes = pd.merge(postulantes_genero_y_edad,postulantes_educacion,how='left',on='idpostulante')
postulantes['educacion'] = postulantes['educacion'].fillna('Ninguna')
postulantes['estado'] = postulantes['estado'].fillna('Ninguno')
postulantes = postulantes.drop(['fechanacimiento'],axis=1)
postulantes.head()

,idpostulante,sexo,edad,educacion,estado,Cod_edu
0,YZP,MASC,57,Master,En Curso,5.0
1,8mW,NO_DECLARA,0,Ninguna,Ninguno,NaN
2,aEWON,FEM,50,Ninguna,Ninguno,NaN
3,Zjde31,MASC,54,Posgrado,Graduado,4.0
4,8wKqoM,FEM,40,Universitario,Graduado,3.0


In [18]:
postulantes.isnull().any()

idpostulante    False
sexo            False
edad            False
educacion       False
estado          False
Cod_edu          True
dtype: bool

## Vistas

In [19]:
#vistas = pd.read_csv('Datos/fiuba_hasta_15_abril/fiuba_3_vistas.csv')

In [20]:
#Vistazo a la estructura del dataframe

#vistas.head()

In [21]:
#El dataframe posee 5758686 registros

#vistas.shape

In [22]:
#No tenemos registros nulos

#vistas.isnull().any()

In [23]:
#En este dataframe tenemos 11223 avisos únicos 

#vistas['idAviso'].value_counts().size

In [24]:
#Y de los postulantes del dataset, 272526 han visto al menos un aviso

#vistas['idpostulante'].value_counts().size

In [25]:
#Modifico el nombre de la columna timestamp y en su lugar pongo un nombre más descriptivo.>

#vistas = vistas.rename(columns = {'timestamp':'fechahoravisualizacion'})

In [26]:
#Ordeno la informacion sobre la fecha y hora de visualizaciones, modificando su formato y agregando columnas que me permitan
#analizar de forma individual la fecha y la hora.

#vistas['fechahoravisualizacion'] = pd.to_datetime(vistas['fechahoravisualizacion'])
#vistas['fechavisualizacion'] = vistas['fechahoravisualizacion'].dt.date
#vistas['horavisualizacion'] = vistas['fechahoravisualizacion'].dt.time
#vistas.sort_values('fechavisualizacion').head(10)

## Postulaciones

In [27]:
postulaciones_hasta_15_abril = pd.read_csv('Datos/fiuba_hasta_15_abril/fiuba_4_postulaciones.csv')
postulaciones_original = pd.read_csv('Datos/datos_navent_fiuba/fiuba_4_postulaciones.csv')

In [28]:
postulaciones = pd.concat([postulaciones_hasta_15_abril,postulaciones_original],axis=0).drop_duplicates()

In [29]:
#Vistazo a la estructura del dataframe

postulaciones.head()

,idaviso,idpostulante,fechapostulacion
0,1112248724,NjlD,2018-01-19 07:39:16
1,1112286523,ZaO5,2018-01-24 15:07:39
2,1112272060,ZaO5,2018-01-24 15:20:10
3,1112288401,ZaO5,2018-01-26 08:37:04
4,1112300563,ZaO5,2018-01-30 13:35:48


In [30]:
#Tenemos 6604534 registros

postulaciones.shape

(6604534, 3)

In [31]:
#No hay registros nulos

postulaciones.isnull().any()

idaviso             False
idpostulante        False
fechapostulacion    False
dtype: bool

## Avisos

In [32]:
#avisos_detalle_desde_15_Abril = pd.read_csv('Datos/Fiuba_desde_15_Abril/fiuba_6_avisos_detalle.csv')
#avisos_online_hasta_15_abril = pd.read_csv('Datos/fiuba_hasta_15_abril/fiuba_5_avisos_online.csv')
#avisos_detalle_hasta_15_abril = pd.read_csv('Datos/fiuba_hasta_15_abril/fiuba_6_avisos_detalle.csv')
#avisos_online_original = pd.read_csv('Datos/datos_navent_fiuba/fiuba_5_avisos_online.csv')
#avisos_detalle_original = pd.read_csv('Datos/datos_navent_fiuba/fiuba_6_avisos_detalle.csv')

In [33]:
#avisos_detalle = pd.concat([avisos_detalle_desde_15_Abril,avisos_detalle_hasta_15_abril,avisos_detalle_original],axis=0).drop_duplicates('idaviso')
#avisos_online = pd.concat([avisos_online_hasta_15_abril,avisos_online_original],axis=0).drop_duplicates('idaviso')

In [34]:
#avisos_online.shape

In [35]:
#avisos_detalle.shape

In [36]:
#No hay registros nulos

#avisos_online.isnull().any()

In [37]:
#avisos_detalle.isnull().any()

# Árbol binario

In [38]:
#Todas las postulaciones historicas

postulaciones_historico = pd.merge(postulantes,postulaciones[['idaviso','idpostulante']],on='idpostulante',how='inner')

In [39]:
postulaciones_historico.Cod_edu = postulaciones_historico.Cod_edu.fillna(7)
postulaciones_historico.head()

,idpostulante,sexo,edad,educacion,estado,Cod_edu,idaviso
0,E5bKKJ,MASC,35,Universitario,Graduado,3.0,1112261798
1,E5bKKJ,MASC,35,Universitario,Graduado,3.0,1112269651
2,E5bKKJ,MASC,35,Universitario,Graduado,3.0,1112341049
3,E5bKKJ,MASC,35,Universitario,Graduado,3.0,1112259997
4,E5bKKJ,MASC,35,Universitario,Graduado,3.0,1112352499


In [40]:
y_train = postulaciones_historico['idaviso']
x_train = postulaciones_historico.drop(['idaviso','estado','educacion','sexo','idpostulante'], axis=1).values 



In [45]:
x_train

array([[35.,  3.],
       [35.,  3.],
       [35.,  3.],
       ...,
       [20.,  2.],
       [20.,  2.],
       [ 0.,  7.]])

In [43]:
y_train.head()

0    1112261798
1    1112269651
2    1112341049
3    1112259997
4    1112352499
Name: idaviso, dtype: int64

In [51]:

# Crear Arbol de decision con profundidad = 4
decision_tree = tree.DecisionTreeClassifier(criterion='entropy',
                                            min_samples_split=20,
                                            min_samples_leaf=5,
                                            max_depth = 4)
decision_tree.fit(x_train, y_train)

DecisionTreeClassifier(class_weight=None, criterion='entropy', max_depth=4,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=5, min_samples_split=20,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best')

In [83]:
#test_final_100k_.head()
y_pred = decision_tree.predict(x_train[0:5260])
print("Prediccion: " + str(y_pred))


Prediccion: [1112094756 1112094756 1112094756 ... 1112094756 1112094756 1112094756]


In [86]:
acc_decision_tree = round(decision_tree.score(x_train[0:5260], y_train[0:5260]) * 100, 2)
print(acc_decision_tree)

0.44
